In [ ]:
import collections
from matplotlib import pyplot as plt
from matplotlib import cm
import pylab
import math

# Open the input file containing DNA sequence data
# Replace with the actual file path for different datasets
f = open("2466139|2020-12-01/2466139|2020-12-01")
# Alternative example: f = open("dados/Hcovid-NL63.fasta")

# Read the content of the file
s1 = f.read()

# Extract the DNA sequence (removing the header if present)
data1 = "".join(s1.split("\n")[1:])  # Join all lines except the first one

# Filter the sequence to keep only valid nucleotide characters (a, c, g, t)
data = ""
for i in range(len(data1)):
    if data1[i] == 'a' or data1[i] == 'c' or data1[i] == 'g' or data1[i] == 't':
        data = data + data1[i]

# Print the length of the original sequence for debugging purposes
print(len(data1))



In [ ]:
# Function to count k-mers (subsequences of length k) in the given DNA sequence
def count_kmers(sequence, k):
    d = collections.defaultdict(int)  # Create a dictionary to count occurrences
    for i in range(len(data) - (k - 1)):  # Iterate over all k-mers in the sequence
        d[sequence[i:i + k]] += 1  # Increment the count for the k-mer
    # Remove k-mers containing "N" (unknown nucleotides)
    for key in list(d):
        if "N" in key:
            del d[key]
    return d

# Function to compute probabilities for each k-mer based on its count
def probabilities(kmer_count, k):
    probabilities = collections.defaultdict(float)  # Store probabilities as floats
    N = len(data)  # Total length of the sequence
    for key, value in kmer_count.items():
        probabilities[key] = float(value) / (N - k + 1)  # Normalize by sequence length
    return probabilities

# Function to create a Chaos Game Representation (CGR) matrix for k-mers
def chaos_game_representation(probabilities, k):
    array_size = int(math.sqrt(4 ** k))  # Determine the size of the matrix (2^k x 2^k)
    chaos = []  # Initialize the matrix
    for i in range(array_size):
        chaos.append([0] * array_size)  # Fill with zeros

    # Initialize variables for CGR coordinate mapping
    maxx = array_size
    maxy = array_size
    posx = 0
    posy = 0

    # Map k-mer probabilities to CGR matrix
    for key, value in probabilities.items():
        for char in key:  # Iterate over each nucleotide in the k-mer
            if char == "g":
                posx += maxx / 2  # Move to the right
            elif char == "c":
                posy += maxy / 2  # Move upward
            elif char == "t":
                posx += maxx / 2  # Move diagonally
                posy += maxy / 2
            maxx /= 2  # Halve the step size
            maxy /= 2
        # Assign the probability value to the appropriate position in the matrix
        chaos[int(posy - 1)][int(posx - 1)] = value
        # Reset variables for the next k-mer
        maxx = array_size
        maxy = array_size
        posx = 1
        posy = 1
    return chaos




In [ ]:
# Define k-mer length
k = 5

# Count k-mers and compute their probabilities
f4 = count_kmers(data, k)
f4_prob = probabilities(f4, k)

# Save the k-mer counts to a text file
with open('5-mers.txt', 'w') as f:
    f.write(str(f4))

# Generate the CGR matrix for the given k
chaos_k4 = chaos_game_representation(f4_prob, k)

# Visualize and save the CGR as an image
pylab.imshow(chaos_k4, interpolation='nearest', cmap=cm.gray_r)
plt.savefig('5mers.png')  # Save the CGR plot as a PNG file
pylab.show()  # Display the CGR plot
